In [38]:
!"{sys.executable}" -m pip install --upgrade pip
!"{sys.executable}" -m pip install numpy
!"{sys.executable}" -m pip install tensorflow

import sys
print(sys.executable)

c:\Users\DESKTOP\.pyenv\pyenv-win\versions\3.10.11\python.exe


In [39]:
import sys
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

MIN_LEN = 1
MAX_LEN = 1000
INPUT_SHAPE = (3, 1)

In [40]:
# 성공/실패 케이스 각각 num_samples 갯수만큼 생성 (gen_triangle_sides)
def gen_triangle_sides(num_samples):
    success_cases = []
    fail_cases = []
    success_cnt = 0
    fail_cnt = 0

    while True:
        # 세 변의 길이를 랜덤으로 생성 (Get three sides by random)
        three_sides = np.random.randint(MIN_LEN, MAX_LEN, INPUT_SHAPE)

        # 가장 긴 변 < 나머지 두 변의 합 : 성공 (the lofailest side < the sum of the others  suceess)
        if is_valid_triangle(three_sides):
            if success_cnt < num_samples:
                success_cases.append(three_sides)
                success_cnt += 1
        else:
            if fail_cnt < num_samples:
                fail_cases.append(three_sides)
                fail_cnt += 1

        # If the data for the entire sample size has been obtained, escape loop
        if success_cnt == num_samples and fail_cnt == num_samples:
            break

    return np.array(success_cases), np.array(fail_cases)

# 세 변이 삼각형을 이룰 수 있는지 체크 (Check whether it is possible to form a triangle with three sides)
def is_valid_triangle(three_sides):
    max_len = max(three_sides)
    others_len = sum(three_sides) - max_len
    return True if others_len > max_len else False


In [41]:
# gen data for train
num_samples = 100000
success_cases, fail_cases = gen_triangle_sides(num_samples)

In [42]:
# 원핫코딩, one hot coding : success = 1, fail = 0
labels = np.concatenate([np.ones(len(success_cases)), np.zeros(len(fail_cases))])

In [43]:
# 인덱스 섞기 (Shuffle data)
success_indices = np.arange(len(success_cases))
fail_indices = np.arange(len(fail_cases))

np.random.shuffle(success_indices)
np.random.shuffle(fail_indices)

# 순서가 섞인 인덱스로 리스트의 값 추출하기
rand_success_cases = success_cases[success_indices]
rand_fail_cases = fail_cases[fail_indices]

# 데이터 정규화 (Data normalization)
norm_success_cases = rand_success_cases / 255.0
norm_fail_cases = rand_fail_cases / 255.0

In [44]:
triangles = np.concatenate([norm_success_cases, norm_fail_cases])

# AI 모델 생성 (Create AI model)
model = models.Sequential([
    layers.Flatten(input_shape=(3, 1)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train Model
model.fit(triangles, labels, epochs=8)

# Evalidate Model
loss, accuracy = model.evaluate(triangles, labels)
print(f'Test Accuracy: {accuracy}')

c:\Users\DESKTOP\.pyenv\pyenv-win\versions\3.10.11\lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/8
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.8967 - loss: 0.2825
Epoch 2/8
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.9834 - loss: 0.0706
Epoch 3/8
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - accuracy: 0.9899 - loss: 0.0443
Epoch 4/8
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - accuracy: 0.9913 - loss: 0.0351
Epoch 5/8
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - accuracy: 0.9927 - loss: 0.0288
Epoch 6/8
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.9938 - loss: 0.0249
Epoch 7/8
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - accuracy: 0.9937 - loss: 0.0226
Epoch 8/8
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - accuracy: 0.9944 - loss: 0.0208
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step - accuracy: 0.9951 - loss: 0.0207
Test Accuracy: 0.9962599873542786


In [45]:
# Example
example = np.array([1,2.1,1])
single_traiangle_coors = np.array([example])
predictions = model.predict(single_traiangle_coors)
predictions

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step


array([[0.00901118]], dtype=float32)

In [46]:
model.save("model.h5")